<a href="https://colab.research.google.com/github/manjunathsirur1994/Time_series_forecasting/blob/main/Regression_for_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import math

In [173]:
ticker = 'LLOYDSME.NS'
df = yf.download(ticker, start='2020-01-01', multi_level_index=False)
df

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2023-07-17 00:00:00+00:00,562.558289,563.299988,582.700012,550.500000,554.000000,417232
2023-07-18 00:00:00+00:00,543.583313,544.299988,573.900024,531.000000,573.900024,284708
2023-07-19 00:00:00+00:00,565.504395,566.250000,572.000000,541.250000,564.700012,208966
2023-07-20 00:00:00+00:00,563.307312,564.049988,572.700012,557.000000,572.700012,108284
2023-07-21 00:00:00+00:00,557.964355,558.700012,564.500000,550.049988,564.500000,189224
...,...,...,...,...,...,...
2024-11-05 00:00:00+00:00,975.500000,975.500000,987.599976,966.950012,970.250000,156433
2024-11-06 00:00:00+00:00,1016.049988,1016.049988,1020.000000,971.150024,971.150024,429220
2024-11-07 00:00:00+00:00,995.750000,995.750000,1030.000000,990.000000,1014.500000,178607


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 326 entries, 2023-07-17 00:00:00+00:00 to 2024-11-11 00:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  326 non-null    float64
 1   Close      326 non-null    float64
 2   High       326 non-null    float64
 3   Low        326 non-null    float64
 4   Open       326 non-null    float64
 5   Volume     326 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 17.8 KB


In [175]:
df = df[['Close']]
df.head()
print(df.shape)

(326, 1)


In [176]:
# Not needed in regression

# df = df.resample('B').last()
# df.fillna(method='bfill')
# df.shape

In [177]:
df.reset_index(inplace=True)
df

,Date,Close
0,2023-07-17 00:00:00+00:00,563.299988
1,2023-07-18 00:00:00+00:00,544.299988
2,2023-07-19 00:00:00+00:00,566.250000
3,2023-07-20 00:00:00+00:00,564.049988
4,2023-07-21 00:00:00+00:00,558.700012
...,...,...
321,2024-11-05 00:00:00+00:00,975.500000
322,2024-11-06 00:00:00+00:00,1016.049988
323,2024-11-07 00:00:00+00:00,995.750000
324,2024-11-08 00:00:00+00:00,1006.049988


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    326 non-null    datetime64[ns, UTC]
 1   Close   326 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 5.2 KB


In [179]:
df['Close_lag1'] = df['Close'].shift(1)
df['Close_lag2'] = df['Close'].shift(2)
df['Close_lag3'] = df['Close'].shift(3)
df['Close_lag4'] = df['Close'].shift(4)
df['Close_lag5'] = df['Close'].shift(5)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['dayofweek'] = df['Date'].dt.dayofweek
df['day_name'] = df['Date'].dt.day_name()
df['quarter'] = df['Date'].dt.quarter
df['dayofyear'] = df['Date'].dt.dayofyear
df['weekofyear'] = df['Date'].dt.isocalendar().week
df['isweekend'] = df['dayofweek'] >= 5
df = df.dropna()
df = df.drop(columns=['Date'])
df

<ipython-input-179-3b646b68015f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close_lag1'] = df['Close'].shift(1)
<ipython-input-179-3b646b68015f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close_lag2'] = df['Close'].shift(2)
<ipython-input-179-3b646b68015f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,Close,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,year,month,day,dayofweek,day_name,quarter,dayofyear,weekofyear,isweekend
5,575.099976,558.700012,564.049988,566.250000,544.299988,563.299988,2023,7,24,0,Monday,3,205,30,False
6,574.900024,575.099976,558.700012,564.049988,566.250000,544.299988,2023,7,25,1,Tuesday,3,206,30,False
7,574.500000,574.900024,575.099976,558.700012,564.049988,566.250000,2023,7,26,2,Wednesday,3,207,30,False
8,575.500000,574.500000,574.900024,575.099976,558.700012,564.049988,2023,7,27,3,Thursday,3,208,30,False
9,596.900024,575.500000,574.500000,574.900024,575.099976,558.700012,2023,7,28,4,Friday,3,209,30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,975.500000,970.150024,978.099976,972.849976,955.750000,950.849976,2024,11,5,1,Tuesday,4,310,45,False
322,1016.049988,975.500000,970.150024,978.099976,972.849976,955.750000,2024,11,6,2,Wednesday,4,311,45,False
323,995.750000,1016.049988,975.500000,970.150024,978.099976,972.849976,2024,11,7,3,Thursday,4,312,45,False
324,1006.049988,995.750000,1016.049988,975.500000,970.150024,978.099976,2024,11,8,4,Friday,4,313,45,False


In [180]:
dummies = pd.get_dummies(df['day_name'])
df = pd.concat([df, dummies], axis=1)
df = df.drop(columns=['day_name'])
df

,Close,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,year,month,day,dayofweek,quarter,dayofyear,weekofyear,isweekend,Friday,Monday,Thursday,Tuesday,Wednesday
5,575.099976,558.700012,564.049988,566.250000,544.299988,563.299988,2023,7,24,0,3,205,30,False,False,True,False,False,False
6,574.900024,575.099976,558.700012,564.049988,566.250000,544.299988,2023,7,25,1,3,206,30,False,False,False,False,True,False
7,574.500000,574.900024,575.099976,558.700012,564.049988,566.250000,2023,7,26,2,3,207,30,False,False,False,False,False,True
8,575.500000,574.500000,574.900024,575.099976,558.700012,564.049988,2023,7,27,3,3,208,30,False,False,False,True,False,False
9,596.900024,575.500000,574.500000,574.900024,575.099976,558.700012,2023,7,28,4,3,209,30,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,975.500000,970.150024,978.099976,972.849976,955.750000,950.849976,2024,11,5,1,4,310,45,False,False,False,False,True,False
322,1016.049988,975.500000,970.150024,978.099976,972.849976,955.750000,2024,11,6,2,4,311,45,False,False,False,False,False,True
323,995.750000,1016.049988,975.500000,970.150024,978.099976,972.849976,2024,11,7,3,4,312,45,False,False,False,True,False,False
324,1006.049988,995.750000,1016.049988,975.500000,970.150024,978.099976,2024,11,8,4,4,313,45,False,True,False,False,False,False


In [181]:
df.replace({True: 1, False: 0}, inplace=True)
df

<ipython-input-181-55e9c1cfbffd>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({True: 1, False: 0}, inplace=True)


,Close,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,year,month,day,dayofweek,quarter,dayofyear,weekofyear,isweekend,Friday,Monday,Thursday,Tuesday,Wednesday
5,575.099976,558.700012,564.049988,566.250000,544.299988,563.299988,2023,7,24,0,3,205,30,0,0,1,0,0,0
6,574.900024,575.099976,558.700012,564.049988,566.250000,544.299988,2023,7,25,1,3,206,30,0,0,0,0,1,0
7,574.500000,574.900024,575.099976,558.700012,564.049988,566.250000,2023,7,26,2,3,207,30,0,0,0,0,0,1
8,575.500000,574.500000,574.900024,575.099976,558.700012,564.049988,2023,7,27,3,3,208,30,0,0,0,1,0,0
9,596.900024,575.500000,574.500000,574.900024,575.099976,558.700012,2023,7,28,4,3,209,30,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,975.500000,970.150024,978.099976,972.849976,955.750000,950.849976,2024,11,5,1,4,310,45,0,0,0,0,1,0
322,1016.049988,975.500000,970.150024,978.099976,972.849976,955.750000,2024,11,6,2,4,311,45,0,0,0,0,0,1
323,995.750000,1016.049988,975.500000,970.150024,978.099976,972.849976,2024,11,7,3,4,312,45,0,0,0,1,0,0
324,1006.049988,995.750000,1016.049988,975.500000,970.150024,978.099976,2024,11,8,4,4,313,45,0,1,0,0,0,0


In [182]:
x = df.drop(columns=['Close'])
y = df['Close']

In [183]:
df.shape

(321, 19)

In [184]:
x_train = x.head(320)
y_train = y.head(320)
x_test = x.tail(1)
y_test = y.tail(1)

In [185]:
y_train.tail(1)

,Close
324,1006.049988


In [186]:
y_test

,Close
325,992.0


In [187]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [188]:
lr.score(x_train, y_train)

0.985278039670889

In [189]:
y_pred = lr.predict(x_test)
y_pred

array([1010.20630544])

In [190]:
y_test - y_pred

,Close
325,-18.206305


In [191]:
y_test

,Close
325,992.0


In [192]:
mean_squared_error(y_test, y_pred)

331.46955783069234

In [193]:
math.sqrt(mean_squared_error(y_test, y_pred))

18.20630544154119

In [194]:
y_train.tail(1)

,Close
324,1006.049988


In [195]:
y_test

,Close
325,992.0


In [196]:
lr.coef_

array([ 9.92046615e-01, -1.42856708e-01,  1.60406688e-01, -5.02571934e-03,
       -4.06724316e-02,  1.22356946e+01, -4.66577359e+01, -1.61747312e+00,
        8.92104735e-01,  1.29787529e+00,  1.31380226e+00,  1.80819888e+00,
       -6.75015599e-14, -1.06137209e+00, -8.01196703e-01,  1.97146647e+00,
        5.59010963e-01, -6.67908643e-01])